<a href="https://colab.research.google.com/github/drfperez/YoungPhotonicsCongress/blob/main/PhotonTunnel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Quantum Tunneling Simulation — Google Colab Version

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display
import ipywidgets as widgets

# ===== Parameters =====
N = 600
dt = 0.1
sigma = 15
x0 = 50
k0 = 0.35
energy = k0**2 / 2
barrierX = 300

# ===== State =====
psiRe = None
psiIm = None
V = None
Tdata = []
Rdata = []

running = False

# ===== Widgets =====
bw_slider = widgets.IntSlider(value=40, min=10, max=150, description='Barrier Width')
bh_slider = widgets.IntSlider(value=5, min=1, max=10, description='Barrier Height')

start_btn = widgets.Button(description='▶ Start')
pause_btn = widgets.Button(description='⏸ Pause')
reset_btn = widgets.Button(description='⏹ Reset')

output = widgets.Output()

# ===== Simulation Functions =====
def initState():
    global psiRe, psiIm, V, Tdata, Rdata
    psiRe = np.zeros(N)
    psiIm = np.zeros(N)
    V = np.zeros(N)

    for i in range(N):
        env = np.exp(-((i-x0)**2)/(2*sigma**2))
        psiRe[i] = env*np.cos(k0*i)
        psiIm[i] = env*np.sin(k0*i)

    updateBarrier()
    Tdata = []
    Rdata = []

def updateBarrier(change=None):
    global V
    V[:] = 0
    w = bw_slider.value
    h = bh_slider.value
    V[barrierX:barrierX+w] = h

bw_slider.observe(updateBarrier, names='value')
bh_slider.observe(updateBarrier, names='value')

def absorb(i):
    d = min(i, N-i)
    return d/40 if d < 40 else 1

def step():
    global psiRe, psiIm

    r = psiRe.copy()
    im = psiIm.copy()

    for i in range(1, N-1):
        lapR = r[i-1] - 2*r[i] + r[i+1]
        lapI = im[i-1] - 2*im[i] + im[i+1]

        psiRe[i] += dt*(-0.5*lapI + V[i]*im[i])
        psiIm[i] -= dt*(-0.5*lapR + V[i]*r[i])

        a = absorb(i)
        psiRe[i] *= a
        psiIm[i] *= a

# ===== Plot Setup =====
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(10,5))

wave_plot, = ax1.plot([],[])
barrier_plot, = ax1.plot([],[])
T_line, = ax2.plot([],[])
R_line, = ax2.plot([],[])

ax1.set_ylim(0,1)
ax1.set_xlim(0,N)
ax1.set_title("Probability Density |ψ|²")

ax2.set_ylim(0,1)
ax2.set_xlim(0,N)
ax2.set_title("Transmission / Reflection")

def update(frame):
    global Tdata, Rdata

    if running:
        step()

    prob = psiRe**2 + psiIm**2

    wave_plot.set_data(range(N), prob*10)
    barrier = np.zeros(N)
    barrier[barrierX:barrierX+bw_slider.value] = bh_slider.value/10
    barrier_plot.set_data(range(N), barrier)

    T = np.sum(prob[barrierX+bw_slider.value:])
    R = np.sum(prob[:barrierX])

    Tdata.append(T)
    Rdata.append(R)

    if len(Tdata)>N:
        Tdata.pop(0)
        Rdata.pop(0)

    T_line.set_data(range(len(Tdata)), Tdata)
    R_line.set_data(range(len(Rdata)), Rdata)

    return wave_plot, barrier_plot, T_line, R_line

ani = FuncAnimation(fig, update, interval=33)

# ===== Controls =====
def start(b):
    global running
    running = True

def pause(b):
    global running
    running = False

def reset(b):
    global running
    running = False
    initState()

start_btn.on_click(start)
pause_btn.on_click(pause)
reset_btn.on_click(reset)

# ===== Run =====
initState()

display(
    widgets.VBox([
        bw_slider,
        bh_slider,
        widgets.HBox([start_btn, pause_btn, reset_btn])
    ])
)

plt.show()
